In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.8/750.8 kB 3.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
!mkdir /content/flask_app

In [ ]:
%cd /content/flask_app

/content/flask_app


In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://nijt7curug-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:
%%writefile app.py

from flask import Flask, render_template, request, send_file, Response
import os
import cv2
from io import BytesIO
import base64
from ultralytics import YOLO
import json

model = YOLO('yolov8n.pt')


app = Flask(__name__)

# 업로드된 파일이 저장될 디렉토리 설정
UPLOAD_FOLDER = 'uploads'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# 디렉토리가 없으면 생성
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

def read_image(file_path):
    # OpenCV로 이미지 읽기
    img = cv2.imread(file_path)
    return img

def encode_image(img):
    # OpenCV 이미지를 base64로 인코딩
    _, buffer = cv2.imencode('.jpg', img)
    img_str = base64.b64encode(buffer).decode('utf-8')
    # print(img_str)
    return img_str

def predict(img):

    results = model(img)     # call method

    for result in results:

        img_shape_list =[]
        boxes_list = []
        im0 = cv2.imencode('.jpg', result.plot())[1].tobytes()
        img_str = base64.b64encode(im0).decode('utf-8')

        orig_img_shape = result[0].orig_img.shape
        orig_shape = result[0].orig_shape

        img_shape_list.append({'orig_img_shape':orig_img_shape, 'orig_shape': orig_shape})

        for boxes in results[0]:

            boxes_cls = boxes.boxes.cls.tolist()
            boxes_xywhn = boxes.boxes.xywhn.tolist()
            boxes_xyxyn = boxes.boxes.xyxyn.tolist()

            # boxes_list.append({'orig_img_shape':orig_img_shape, 'orig_shape': orig_shape,'boxes_cls':boxes_cls, 'boxes_xywhn':boxes_xywhn, 'boxes_xyxyn':boxes_xyxyn})
            boxes_list.append({'boxes_cls':boxes_cls, 'boxes_xywhn':boxes_xywhn, 'boxes_xyxyn':boxes_xyxyn})
            # print(boxes_list)


        # yield (b'--frame\r\n'
        #         b'Content-Type: image/jpeg\r\n\r\n' + im0 + b'\r\n')
        return img_str, boxes_list, img_shape_list


@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload():
    # HTML 폼에서 'file' 필드로 전송된 파일을 가져옴
    file = request.files['file']

    if file:
        # 업로드된 파일을 저장할 경로 생성
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)

        # 파일이 존재하는지 확인
        # if os.path.isfile(filepath):
        #     return "동일한 파일이 이미 존재합니다."

        # 파일 저장
        file.save(filepath)

        # 이미지 파일을 OpenCV로 읽기
        img = read_image(filepath)

        # HTML에 전달할 base64로 인코딩된 이미지
        img_str = encode_image(img)

        predict_result =  predict(img)

        # 이미지 파일 경로와 인코딩된 이미지를 HTML에 전달하여 표시
        return render_template('uploaded.html', filepath=filepath, img_str=img_str, predictimg = predict_result[0], data=predict_result[1], shape=predict_result[2])
        # return render_template('uploaded.html', filepath=filepath, img_str=img_str, predictimg = predict_result[0], data=predict_result[1])
        # return render_template('uploaded.html', filepath=filepath, img_str=img_str, predictimg = predict_result)


    return "파일 업로드에 실패했습니다."


if __name__ == '__main__':
    # 서버 실행
    app.run(debug=True)


Writing app.py


In [ ]:
!mkdir /content/flask_app/templates

In [ ]:
%cd /content/flask_app/templates

/content/flask_app/templates


In [ ]:
%%writefile index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>이미지 입력, 출력 및 업로드 예제</title>
</head>
<body>

    <h1>이미지 입력, 출력 및 업로드 예제</h1>

    <!-- 이미지 입력 폼
    <form id="imageForm" action="/upload" method="post" enctype="multipart/form-data">
        <input type="file" name="file" accept="image/*">
        <input type="button" value="이미지 출력" onclick="displayImage()">
        <input type="submit" value="업로드">
    </form>
    -->

    <form id="imageForm" action="/upload" method="post" enctype="multipart/form-data">
        <input type="file" id="file" name="file" accept="image/*" onchange="displayImage()">
        <!--- input type="button" value="이미지 출력" onclick="displayImage()" --->
        <input type="submit" value="predict">
    </form>
    <!-- id   : id 속성은 해당 요소의 고유한 식별자(identifier)를 제공, javascript, css의 식별자
         name : name 속성은 폼(form)을 서버로 제출할 때 해당 입력 필드의 이름을 나타냄,
                서버 측에서는 이 이름을 사용하여 사용자가 입력한 데이터를 식별하고 처리함
                여러 입력 필드가 한 폼 안에 있을 때, 각 입력 필드에 고유한 name을 부여하여 서버에서 구별-->


    <!-- 선택된 이미지 출력 -->
    <div id="outputContainer">
        <img id="selectedImage" src="#" alt="선택된 이미지">
    </div>

    <script>
        // 이미지 출력 함수
        function displayImage() {
            var input = document.querySelector('input[type="file"]');
            var selectedImage = document.getElementById('selectedImage');

            if (input.files && input.files[0]) {
                var reader = new FileReader();

                reader.onload = function (e) {
                    selectedImage.src = e.target.result;
                };
                reader.readAsDataURL(input.files[0]);
            }
        }
    </script>

</body>
</html>

Writing index.html




```
        <input type="file" name="file" accept="image/*">


type="file": 이 속성은 입력 필드의 타입을 나타냅니다. file 타입은 파일을 선택할 수 있는 파일 입력 필드를 생성합니다.

name="file": 이 속성은 입력 필드의 이름을 정의합니다. 서버에서 해당 이름을 사용하여 파일을 전송할 때 식별할 수 있습니다.

accept="image/*": 이 속성은 허용되는 파일의 종류를 지정합니다. 여기에서는 이미지 파일만 허용하도록 지정되어 있습니다. image/*는 모든 이미지 파일 형식을 허용합니다. 사용자는 파일 선택 창에서 이미지 파일만 선택할 수 있게 됩니다.
```



In [ ]:
%%writefile uploaded.html

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>업로드 완료</title>
</head>
<body>
    <h1>업로드 완료</h1>
    <img src="data:image/jpeg;base64,{{ img_str }}" alt="Uploaded Image"width="640" height="480"  >
    <img src="data:image/jpeg;base64,{{ predictimg }}" alt="Uploaded Image" width="640" height="480" >

    <br>
    <a href="/" class='btn'>go back</a>
    <br>

    {% for item in  shape %}
    <li>orig_img_shape: {{ item.orig_img_shape }}</li>
    <li>orig_shape: {{ item.orig_shape}}</li>
    <br>
    {% endfor %}

    {% for item in  data %}

        <li>boxes_cls: {{item.boxes_cls}}</li>
        <li>boxes_xywhn: {{item.boxes_xywhn}}</li>
        <li>boxes_xyxyn: {{ item.boxes_xyxyn }}</li>
        <br>
    {% endfor %}
</body>
</html>

Writing uploaded.html


In [ ]:
%cd ..

/content/flask_app


In [ ]:
!flask run

100% 6.23M/6.23M [00:00<00:00, 75.9MB/s]
 * Debug mode: off
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [23/Apr/2024 00:45:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2024 00:45:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2024 00:45:41] "GET /favicon.ico HTTP/1.1" 404 -

0: 640x640 1 cat, 458.0ms
Speed: 33.2ms preprocess, 458.0ms inference, 3018.9ms postprocess per image at shape (1, 3, 640, 640)
127.0.0.1 - - [23/Apr/2024 00:46:02] "POST /upload HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2024 00:46:03] "GET /favicon.ico HTTP/1.1" 404 -
